In [1]:
import os
from fastai.text import *
from fastai import *
import pandas as pd
import numpy as np
from numba import jit
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import seaborn as sns

In [3]:
## load the training set databunch and the respective encoder 
os.chdir("D:/fast_ai/NLP/cap/encoder/new_extraction")
path = Path("D:/fast_ai/NLP/cap/encoder/new_extraction")
all_train = pd.read_pickle("all_df.pkl") ## here text and train is combined 
enc = pd.read_pickle("encX.pkl") ## this is the encoding matrix from language model 
fastai_bunch = load_data(path,"tran_test") ## data in fastai text format to extract numbers referring to look up in encoder matrix

In [4]:
dataO = fastai_bunch.train_ds.items  ## here: numbers for vocabs - each review has array of number respresenting the words
## each word can then be looked up in encoder array

In [5]:
## wrapper to select subset of data 
def select_subset(train_df,col="product_title",conditions="Candy Crush Saga"):
    return np.where(train_df[col] == conditions)[0]


In [6]:
### 1) created matrix of all words in review,i.e. number words X 400
###2 ) calulates average 
##3) returns matrix n_revies X400
## might take some time to run, because numba does not work due to some incompatibility 
def create_review_embedding(dataO,enc):
    outvec = np.zeros(400)
    
    for i in dataO:
         outvec = np.vstack((outvec,enc[i].mean(axis=0)))
    
    return np.delete(outvec,(0),axis=0)

In [ ]:
vecs = create_review_embedding(dataO,enc.values)

In [ ]:
pd.DataFrame(vecs).to_pickle("vecS.pkl")


In [7]:
vecs = pd.read_pickle("vecS.pkl")

In [8]:
### function to create cluters and to return reviews closest to cluster center

def highlight_most_representative_reviews(vecs,n_reviews=None,ixx=vecs.index):
    vecS = vecs.iloc[ixx,:].copy()
    if n_reviews is None:
        n_reviews = np.int(vecS.shape[0]**0.5)
    km = KMeans(n_clusters=n_reviews)
    km = km.fit(vecS)
    closeness,_ = pairwise_distances_argmin_min(km.cluster_centers_,vecS)
    return (closeness,km.labels_)



In [10]:
### make an example with product candy saga game 



candy_index = select_subset(all_train)

ix_close, cluster_labels = highlight_most_representative_reviews(vecs,n_reviews=10,ixx=candy_index)

candy_data = all_train.iloc[candy_index,:].copy()

most_rep = candy_data.iloc[ix_close,:]

candy_data["cluster"] = cluster_labels




In [ ]:
## print out most reprsentative reviews
file= open("candy_saga_2.txt","w")
for i in most_rep["reviews.text"]:
    file.write("\n"+i)
file.close()
